In [2]:
import pandas as pd
import itertools
import ast

In [3]:
def select_input_id(df, n, k):
    selected_samples = (df
        .groupby(["test_set", "input_id"]).first()
        .groupby("test_set").sample(n*k)
        .groupby("test_set")
    )
    
    indexes = []
    for i in range(0, n*k, n):
        index = [
            list(group.index[i:i+n])
            for _, group in selected_samples            
        ]
        index = list(itertools.chain(*index))
        index = pd.MultiIndex.from_tuples(index)
        indexes.append(index)
        
        
    return indexes

In [4]:
def get_annotator_examples(texts, model, test_set_input_ids):
    texts = texts[texts.index.get_level_values("model").isin(MODELS)]
    examples = []
    for idx in test_set_input_ids:
        exp = texts.xs(idx, level=["test_set", "input_id"], drop_level=False)
        examples.append(exp)
    return pd.concat(examples)

In [31]:
texts_df = (
    pd.read_csv("./data/texts.csv", index_col=[0])
    .set_index(["model", "test_set", "input_id"], append=True)
)
texts_df.index = texts_df.index.set_levels(
    [[ast.literal_eval(x) if x[0] == "(" else x for x in texts_df.index.levels[1]],],
    level=[1]
)

In [34]:
texts_df.xs(656)

,,,inputs,value
model,test_set,input_id,,
"(llamaVIII, lora, etrfr)",test_etr_fr,21,Profitant d'un moment où les braconniers rient...,Lucas se faufile dans le camion des braconnier...


In [120]:
texts_df.index.get_level_values("model").unique()

Index([('llamaVIIIinst', 'rag', 'etrfr+orangesum+wikilarge'),
                     ('llamaVIIIinst', 'zero_shot', 'etrfr'),
                                                    'labels',
                 ('llamaVIII', 'mtllora', 'etrfr+wikilarge'),
                           ('llamaVIIIinst', 'cot', 'etrfr'),
                       ('mistVIIinst', 'zero_shot', 'etrfr'),
                 ('llamaVIII', 'mtllora', 'etrfr+orangesum'),
                              ('llamaVIII', 'lora', 'etrfr'),
                   ('mistVIIinst', 'rag', 'etrfr+wikilarge'),
                             ('mistVIIinst', 'rag', 'etrfr'),
                 ('llamaVIIIinst', 'rag', 'etrfr+wikilarge'),
                   ('mistVIIinst', 'rag', 'etrfr+orangesum'),
                                ('mistVII', 'lora', 'etrfr'),
                           ('llamaVIIIinst', 'rag', 'etrfr'),
                 ('llamaVIIIinst', 'rag', 'etrfr+orangesum'),
                             ('mistVIIinst', 'cot', 'etrfr'),
        

# One DF / Annotator

In [8]:
MODELS = [
    (    'llamaVIII', 'lora', 'etrfr'),
    (    'llamaVIII', 'mtllora', 'etrfr+orangesum+wikilarge'),
    ('llamaVIIIinst',     'rag',                     'etrfr'),
    ('llamaVIIIinst',     'rag',           'etrfr+wikilarge')
]
N_SAMPLES = 3
N_ANNOTATORS = 10

In [9]:
INPUT_IDS = select_input_id(texts_df, N_SAMPLES, N_ANNOTATORS)

In [10]:
assert texts_df[texts_df.index.get_level_values("model").isin(MODELS)].shape[0]/4-53 == 33

In [11]:
get_annotator_examples(texts_df, MODELS, INPUT_IDS[0])

,,,,inputs,value
,model,test_set,input_id,,
59,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr,26,"Soudain, le plus âgé des deux braconniers lève...","Soudain, les 2 braconniers regardent le ciel. ..."
489,"(llamaVIII, lora, etrfr)",test_etr_fr,26,"Soudain, le plus âgé des deux braconniers lève...",Les deux braconniers voient quelque chose dans...
1091,"(llamaVIIIinst, rag, etrfr+wikilarge)",test_etr_fr,26,"Soudain, le plus âgé des deux braconniers lève...",Le plus âgé des deux braconniers montre le cie...
1607,"(llamaVIIIinst, rag, etrfr)",test_etr_fr,26,"Soudain, le plus âgé des deux braconniers lève...",Les deux braconniers regardent le ciel. Ils on...
57,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr,24,« Le sorcier m'a indiqué le camion des braconn...,« Le sorcier m'a indiqué le camion des braconn...
487,"(llamaVIII, lora, etrfr)",test_etr_fr,24,« Le sorcier m'a indiqué le camion des braconn...,« Le sorcier m'a indiqué le camion des braconn...
1089,"(llamaVIIIinst, rag, etrfr+wikilarge)",test_etr_fr,24,« Le sorcier m'a indiqué le camion des braconn...,Manon parle à Lucas de son aventure avec les b...
1605,"(llamaVIIIinst, rag, etrfr)",test_etr_fr,24,« Le sorcier m'a indiqué le camion des braconn...,Manon raconte à Lucas qu'elle a vu le camion d...
40,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr,7,Lucas enlève ses chaussures et se couche sur l...,Lucas se couche sur le lit de son amie. Il lit...


In [109]:
for i, idx in enumerate(INPUT_IDS):
    df = get_annotator_examples(texts_df, MODELS, idx)
    df.to_csv(f"./data/annot_dfs/samples_ann_{i+1}.csv")

# Control DF

In [172]:
MODELS = [
    (    'llamaVIII', 'mtllora', 'etrfr+orangesum+wikilarge'),
    ('llamaVIIIinst',     'rag',                     'etrfr'),
    "labels"
]
N_SAMPLES = 20
N_ANNOTATORS = 1

In [173]:
INPUT_IDS = select_input_id(texts_df, N_SAMPLES, N_ANNOTATORS)

In [174]:
df = get_annotator_examples(texts_df, MODELS, INPUT_IDS[0])

In [175]:
df

,,,,inputs,value
,model,test_set,input_id,,
206,labels,test_etr_fr,1,"En arrivant chez lui, Lucas jette son cartable...",Lucas rentre dans sa maison. Il goûte. Soudain...
1152,"(llamaVIIIinst, rag, etrfr)",test_etr_fr,1,"En arrivant chez lui, Lucas jette son cartable...",Lucas arrive chez lui et se repose sur le cana...
1582,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr,1,"En arrivant chez lui, Lucas jette son cartable...",Lucas arrive chez lui. Il jette son cartable d...
240,labels,test_etr_fr,35,Flairtout se sent de trop. Il s'éloigne d'eux ...,"« Luna, décollage! Nous rentrons. Notre missio..."
1186,"(llamaVIIIinst, rag, etrfr)",test_etr_fr,35,Flairtout se sent de trop. Il s'éloigne d'eux ...,Flairtout est de trop. Il s'éloigne. une souco...
...,...,...,...,...,...
1130,"(llamaVIIIinst, rag, etrfr)",test_etr_fr_politic,12,"ENSEMBLE, NOUS RELÈVERONS LE DÉFI DÉMOCRATIQUE...",Je veux rendre le pouvoir aux citoyens. J'aime...
1560,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr_politic,12,"ENSEMBLE, NOUS RELÈVERONS LE DÉFI DÉMOCRATIQUE...",Je veux rendre le pouvoir aux citoyens de notr...
174,labels,test_etr_fr_politic,2,AUGMENTATION DES SALAIRES ET DES PENSIONS! Les...,"Il faut augmenter les salaires, les pensions e..."


In [204]:
(df
    .reset_index()
    .groupby(["test_set", "model"])
    .apply(lambda g: g.sample(n=1))
    .reset_index(drop=True)
    .set_index(["level_0", "model", "test_set", "input_id"])
).to_csv(f"./data/annot_dfs/inter_annotator.csv")

/tmp/ipykernel_2085602/1720060539.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=1))
